In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")
base_url

'https://api.deepseek.com'

In [2]:
# Chat接口
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        { "role": "system", "content": "你是一个大模型专业的AI助手" },
        { "role": "user", "content": "请解释一下Transformer的核心机制" },
    ]
)
response

ChatCompletion(id='fa7d2d88-19e4-4c50-a246-4518319ce1e9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='好的，Transformer 的核心机制可以概括为：**一种完全基于自注意力机制和前馈神经网络，摒弃了循环和卷积结构，用于处理序列数据的模型架构。**\n\n它的核心思想是：**让序列中的每一个元素（如一个词）都能够直接“看到”并聚合整个序列中所有其他元素的信息**，从而进行高效的并行计算和长距离依赖建模。\n\n下面我们来分解其最核心的几个机制：\n\n### 1. 自注意力机制\n这是 Transformer 的灵魂。它的目的是计算序列中每个元素与其他所有元素的“关联度”（注意力分数），然后根据这个关联度对所有元素的信息进行加权求和，从而得到该元素新的、包含全局信息的表示。\n\n**计算过程（以“我 爱 你”这句话为例）：**\n1.  **生成三个向量**：对于输入序列中的每个词（如“爱”），通过可学习的权重矩阵，生成三个向量：\n    *   **查询向量**：代表“我要寻找什么信息”。\n    *   **键向量**：代表“我拥有什么信息，可供匹配”。\n    *   **值向量**：代表“我实际要提供的信息”。\n2.  **计算注意力分数**：用“爱”的 **Q** 去和序列中所有词（包括“爱”自己）的 **K** 做点积，得到一组分数。分数越高，表示“爱”与那个词在当前任务下的关联性越强。\n    *   例如，“爱”的 Q 与“我”的 K 点积可能得到一个分数，与“你”的 K 点积得到另一个分数。\n3.  **缩放与归一化**：将分数除以一个缩放因子（通常是键向量维度的平方根），然后通过 Softmax 函数进行归一化，得到一组权重（和为1）。这确保了梯度的稳定性。\n4.  **加权求和**：用上一步得到的权重，对序列中所有词的 **V** 进行加权求和。最终得到的向量就是“爱”这个词经过自注意力机制处理后的新表示。**这个新表示融合了它与整个句子中所有词的关系信息。**\n\n**多头注意力**：为了捕捉不同种类的关系（

In [3]:
# 流式响应
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

stream = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        { "role": "system", "content": "你是一个大模型专业的AI助手" },
        { "role": "user", "content": "请分点简述注意力机制的核心思想" },
    ],
    stream=True,
)
for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="", flush=True)

注意力机制的核心思想是通过动态分配权重，让模型在处理信息时能够聚焦于最相关的部分。以下是其核心思想的分点简述：

1. **选择性聚焦**  
   模仿人类注意力，从大量输入信息中筛选出关键部分，忽略无关或次要内容，提高信息处理效率。

2. **权重动态分配**  
   通过计算查询（Query）与键（Key）的相似度，生成注意力权重，值（Value）根据权重加权聚合，实现动态重要性调整。

3. **上下文感知**  
   结合输入序列的全局信息，为每个位置生成上下文相关的表示，克服传统序列模型（如RNN）的长距离依赖问题。

4. **并行化与可扩展性**  
   自注意力机制允许同时计算所有位置的关系，支持并行计算，适用于长序列处理，并易于扩展至多头注意力。

5. **通用适配性**  
   可灵活嵌入不同神经网络架构（如Transformer、CNN、RNN），广泛应用于自然语言处理、计算机视觉、多模态任务等领域。

这些思想使注意力机制成为现代深度学习模型（如GPT、BERT）的核心组件，显著提升了模型的表达能力和性能。

In [4]:
# 函数调用模式，结构化任务执行能力
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        { "role": "user", "content": "请帮我查询北京的天气" }
    ],
    functions=[
        {
            "name": "get_weather",
            "description": "查询天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名称",
                    }
                },
                "required": ["city"],
            }
        }
    ],
    function_call="auto",
)
# 函数调用结构
print(response.choices[0].message.function_call)
print(response.choices[0].message.content)

None
我目前无法直接查询实时天气数据。不过，您可以通过以下方式快速获取北京的天气信息：

1. **推荐使用**：
   - 打开手机天气应用（如苹果天气、墨迹天气等）
   - 在搜索引擎（百度/谷歌）直接搜索“北京天气”
   - 访问中国天气网（www.weather.com.cn）

2. **当前季节提示**（2024年1月）：
   - 北京正处于冬季，近期气温通常在-5℃~5℃之间
   - 建议关注：空气湿度、风力及空气质量指数（AQI）

3. **如需出行建议**：
   - 冬季需准备羽绒服、围巾等防寒衣物
   - 雾霾天气建议佩戴防护口罩

需要我帮您整理其他城市的气候特点或出行贴士吗？


In [ ]:
# 多轮上下文控制模式
from openai import OpenAI

history = [
    { "role": "system", "content": "你是一个编程助手" },
    { "role": "user", "content": "写一个快速排序函数" },
    { "role": "assistant", "content": "好的，以下是快速排序的python代码..." },
    { "role": "user", "content": "请说明时间复杂度" },
]

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=history,
    temperature=0.5,
    max_tokens=150,
)

response.choices[0].message.content

'好的，快速排序的时间复杂度分析如下：\n\n### 1. 最佳情况和平均情况：O(n log n)\n*   **最佳情况**：每次选择的基准（pivot）都能将数组**均匀地**分成两个大小大致相等的子数组。\n    *   在这种情况下，递归树的深度约为 **log₂ n**（以2为底n的对数）。\n    *   在递归树的每一层，所有子问题的划分操作（`partition`）加起来都需要遍历整个数组，总时间为 **O(n)**。\n    *   因此，总时间复杂度为 **O(n) * O(log n) = O(n log n)**。\n*   **平均情况**：对于一个随机排列的输入数组，快速排序的平均时间复杂度'